# Bank Customer Churn Prediction

Use logistic regression to predict churn

* Dataset: https://www.kaggle.com/datasets/gauravtopre/bank-customer-churn-dataset

In [103]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

In [104]:
df = pd.read_csv('customer-churn-records.csv')

df.columns = df.columns.str.lower().str.replace(' ', '_')

categorical_columns = list(df.dtypes[df.dtypes == 'object'].index)

for c in categorical_columns:
    df[c] = df[c].str.lower().str.replace(' ', '_')

In [105]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)

In [106]:
numerical = [
            'credit_score',
            'age',
            'tenure',
            'balance',
            'products_number',
            'credit_card',
            'active_member',
            'estimated_salary'
        ]

categorical = [
        'country',
        'gender'
    ]

In [107]:
def train(df_train, y_train, C=1.0):
    dicts = df_train[categorical + numerical].to_dict(orient='records')

    dv = DictVectorizer(sparse=False)
    X_train = dv.fit_transform(dicts)

    model = LogisticRegression(C=C, max_iter=1000, solver='liblinear', class_weight='balanced')
    model.fit(X_train, y_train)
    
    return dv, model


In [108]:
def predict(df, dv, model):
    dicts = df[categorical + numerical].to_dict(orient='records')

    X = dv.transform(dicts)
    y_pred = model.predict_proba(X)[:, 1]

    return y_pred

In [109]:
C = 1.0
n_splits = 5

In [110]:
kfold = KFold(n_splits=n_splits, shuffle=True, random_state=1)

scores = []

for train_idx, val_idx in kfold.split(df_full_train):
    df_train = df_full_train.iloc[train_idx]
    df_val = df_full_train.iloc[val_idx]

    y_train = df_train.churn.values
    y_val = df_val.churn.values


    dv, model = train(df_train, y_train, C=C)
    y_pred = predict(df_val, dv, model)

    auc = roc_auc_score(y_val, y_pred)
    scores.append(auc)

print('C=%s %.3f +- %.3f' % (C, np.mean(scores), np.std(scores)))

C=1.0 0.737 +- 0.023


In [111]:
scores

[np.float64(0.7093175853018372),
 np.float64(0.7131032758189547),
 np.float64(0.7537632344262178),
 np.float64(0.7367352364073676),
 np.float64(0.7696151496074656)]

In [112]:
dv, model = train(df_full_train, df_full_train.churn.values, C=1.0)
y_pred = predict(df_test, dv, model)

y_test = df_test.churn.values
auc = roc_auc_score(y_test, y_pred)
auc

np.float64(0.7475732583330167)

In [113]:
import pickle


Save the model

In [114]:
output_file = f'model_C={C}.bin'

In [115]:
output_file

'model_C=1.0.bin'

In [116]:
f_out = open(output_file, 'wb') 
pickle.dump((dv, model), f_out)
f_out.close()

In [117]:
!ls -lh *.bin

-rw-r--r--  1 senalidilumika  staff   1.2K Nov 24 01:18 model_C=1.0.bin


In [118]:
with open(output_file, 'wb') as f_out: 
    pickle.dump((dv, model), f_out)

Load the model

In [119]:
import pickle

In [120]:
input_file = 'model_C=1.0.bin'

In [121]:
with open(input_file, 'rb') as f_in: 
    dv, model = pickle.load(f_in)

In [122]:
model

LogisticRegression(class_weight='balanced', max_iter=1000, solver='liblinear')

In [123]:
customer = {
 'country': 'germany',
 'gender': 'male',
 'credit_score': 655,
 'age': 52,
 'tenure': 9,
 'balance': 144696.75,
 'products_number': 1,
 'credit_card': 1,
 'active_member': 1,
 'estimated_salary': 49025.79
 }

In [124]:
X = dv.transform([customer])

In [125]:
y_pred = model.predict_proba(X)[0, 1]

In [126]:
print('input:', customer)
print('output:', y_pred)

input: {'country': 'germany', 'gender': 'male', 'credit_score': 655, 'age': 52, 'tenure': 9, 'balance': 144696.75, 'products_number': 1, 'credit_card': 1, 'active_member': 1, 'estimated_salary': 49025.79}
output: 0.683151062071004


Making requests

In [127]:
import requests

In [128]:
url = 'http://localhost:9696/predict'

In [129]:
customer = {
 'country': 'germany',
 'gender': 'male',
 'credit_score': 655,
 'age': 52,
 'tenure': 9,
 'balance': 144696.75,
 'products_number': 1,
 'credit_card': 1,
 'active_member': 1,
 'estimated_salary': 49025.79
 }

In [130]:
response = requests.post(url, json=customer).json()
response

ConnectionError: HTTPConnectionPool(host='localhost', port=9696): Max retries exceeded with url: /predict (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x173a358e0>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [ ]:
if response['churn']:
    print('sending email to', 'asdx-123d')

NameError: name 'response' is not defined